# Dataset Prompting

Dataset considered in this notebook = 8,9,10,11,12,13
We try to follow author et al. as close as possible. 

- Datasets 8 and 9 are evaluated on the dev set.
- For Datasets 10 and 11, author et al. did not release the code for prompting. So we use the data they reported on their Excel file.
- Dataset 12 is in Polish, so we exclude it from our research.
- For Dataset 13, from the notebooks released by author et al., it seems that they take the first 500 examples. However, on their Excel file, they report 1000 rows. So we follow the Excel file.

In [4]:
import pandas as pd
from numpy import nan

# Dataset 8

This dataset is about entailment. That is, given a premise and a hypothesis a software should be able to tell if the hypothesis logically follows from the premise. A sort of Aristotelian syllogism.

In [73]:
data = pd.read_excel("ChatGPT.xlsx", sheet_name=8)

In [74]:
data.head()

,Unnamed: 0,premise,hypothesis,label,idx,split,prompt,id_check,chatgpt_raw,chatgpt_final
0,0,No Weapons of Mass Destruction Found in Iraq Yet.,Weapons of Mass Destruction Found in Iraq.,not_entailment,0,train,"Having premise ""No Weapons of Mass Destruction...",0.0,not_entailment,NaN
1,1,"A place of sorrow, after Pope John Paul II die...",Pope Benedict XVI is the new leader of the Rom...,entailment,1,train,"Having premise ""A place of sorrow, after Pope ...",1.0,entailment,NaN
2,2,Herceptin was already approved to treat the si...,Herceptin can be used to treat breast cancer.,entailment,2,train,"Having premise ""Herceptin was already approved...",2.0,entailment,NaN
3,3,"Judie Vivian, chief executive at ProMedica, a ...",The previous name of Ho Chi Minh City was Saigon.,entailment,3,train,"Having premise ""Judie Vivian, chief executive ...",3.0,entailment,NaN
4,4,A man is due in court later charged with the m...,Paul Stewart Hutchinson is accused of having s...,not_entailment,4,train,"Having premise ""A man is due in court later ch...",4.0,not_entailment,NaN


In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5767 entries, 0 to 5766
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     5767 non-null   int64  
 1   premise        5767 non-null   object 
 2   hypothesis     5767 non-null   object 
 3   label          2767 non-null   object 
 4   idx            5767 non-null   int64  
 5   split          5767 non-null   object 
 6   prompt         5767 non-null   object 
 7   id_check       5267 non-null   float64
 8   chatgpt_raw    5267 non-null   object 
 9   chatgpt_final  0 non-null      float64
dtypes: float64(2), int64(2), object(6)
memory usage: 450.7+ KB


In [76]:
template = "Your task is to tell, given a premise and a hypothesis, if the hypothesis logically follows from the premise. I will give you the following examples to guide you:\n\n{training_examples}\n\n\
Give your answer for the following sentence in just a python list without elaboration. Only one answer is admissible. Answer with \"entailment\" if the hypothesis follows from the premise and \"not_entailment\" otherwise.\
\npremise: {test_premise} hypothesis: {test_hypothesis}"

In [77]:
print(template)

Your task is to tell, given a premise and a hypothesis, if the hypothesis logically follows from the premise. I will give you the following examples to guide you:

{training_examples}

Give your answer for the following sentence in just a python list without elaboration. Only one answer is admissible. Answer with "entailment" if the hypothesis follows from the premise and "not_entailment" otherwise.
premise: {test_premise} hypothesis: {test_hypothesis}


In [364]:
def stratified_sample_df(df, col, n_samples, random_state = 42):
    n = min(n_samples, df[col].value_counts().min())
    df_ = df.groupby(col).apply(lambda x: x.sample(n, random_state = random_state))
    df_.index = df_.index.droplevel(0)
    return df_

In [112]:
# Filtering out data with no label or not in training set
filtered = data[pd.notna(data['label']) & (data['split'] == 'train')][['premise','hypothesis','label']]

# Sampling training examples (20 examples, 10 per class)
training_examples = stratified_sample_df(filtered, 'label', 10)
training_examples

,premise,hypothesis,label
1364,Yet fishermen from both sides say boats lack n...,There is a territorial waters dispute.,entailment
2097,The settlement must be approved by Citigroup's...,The University of California's board of Regent...,entailment
1820,A Belgian policeman posing as an art dealer in...,Three Swedes were arrested in a Belgian police...,entailment
497,"The high-speed train, scheduled for a trial ru...",The train accelerates to 430 kilometers per hour.,entailment
669,"Brent Frei, the co-founder of midmarket CRM ve...",Brent Frei quits ceo position.,entailment
943,White supported Gough Whitlam's Labor governme...,White assisted the Labor government from 1972 ...,entailment
517,"Qin Shi Huang, personal name Zheng, was king o...",Qin Shi Huang was the first China Emperor.,entailment
170,Thousands of Shias have marched in the souther...,Thousands of Shiite Muslims rallied in the sou...,entailment
86,"On April 28 1945, at the Piazzale Loreto in Mi...",Mussolini's mistress was Clara Petacci.,entailment
2482,"Cyrus captured Babylon without a battle, and r...",Babylon surrendered to Cyrus without going to ...,entailment


In [141]:
# Test set = dev split, as done by author et al. paper
test = data[pd.notna(data['label']) & (data['split'] == 'dev')][['premise','hypothesis','label']]
test

,premise,hypothesis,label
2490,"Dana Reeve, the widow of the actor Christopher...",Christopher Reeve had an accident.,not_entailment
2491,"Yet, we now are discovering that antibiotics a...",Bacteria is winning the war against antibiotics.,entailment
2492,Cairo is now home to some 15 million people - ...,15 million tonnes of rubbish are produced dail...,not_entailment
2493,"The Amish community in Pennsylvania, which num...",Pennsylvania has the biggest Amish community i...,not_entailment
2494,Security forces were on high alert after an el...,Security forces were on high alert after a cam...,entailment
...,...,...,...
2762,Swansea striker Lee Trundle has negotiated a l...,Lee Trundle is in business with the League One...,entailment
2763,A light blue 1975 Ford Escort GL once owned by...,A Houston multimillionaire buys the Pope's For...,entailment
2764,"On October 1 2001, EU and other countries intr...","In 2001, the EU introduced a passport for pets.",entailment
2765,Kozlowski and the company's former chief finan...,"Kozlowski was sentenced, Monday, to serve up t...",entailment


In [142]:
def format_training_example(premise, hypothesis, answer):
    return f"- Premise: \"{premise}\" hypothesis: \"{hypothesis}\" answer: \"{answer}\""

def make_prompt(training_examples, test_premise, test_hypothesis):
    return f"Your task is to tell, given a premise and a hypothesis, if the hypothesis logically follows from the premise. I will give you the following examples to guide you:\n\n{training_examples}\n\n\
Give your answer for the following sentence in just a python list without elaboration. Only one answer is admissible. Answer with \"entailment\" if the hypothesis follows from the premise and \"not_entailment\" otherwise.\
\npremise: {test_premise} hypothesis: {test_hypothesis}"

for k in range(1, 11):
    few_shot_examples = stratified_sample_df(training_examples, 'label', k).apply(lambda x: format_training_example(x['premise'], x['hypothesis'], x['label']), axis = 1)
    examples_string = "\n".join(list(few_shot_examples))
    column_name = 'Prompt for ' + str(k) + '-shot'
    test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['premise'], x['hypothesis']), axis = 1)

test.head()

,premise,hypothesis,label,Prompt for 1-shot,Prompt for 2-shot,Prompt for 3-shot,Prompt for 4-shot,Prompt for 5-shot,Prompt for 6-shot,Prompt for 7-shot,Prompt for 8-shot,Prompt for 9-shot,Prompt for 10-shot
2490,"Dana Reeve, the widow of the actor Christopher...",Christopher Reeve had an accident.,not_entailment,"Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy..."
2491,"Yet, we now are discovering that antibiotics a...",Bacteria is winning the war against antibiotics.,entailment,"Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy..."
2492,Cairo is now home to some 15 million people - ...,15 million tonnes of rubbish are produced dail...,not_entailment,"Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy..."
2493,"The Amish community in Pennsylvania, which num...",Pennsylvania has the biggest Amish community i...,not_entailment,"Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy..."
2494,Security forces were on high alert after an el...,Security forces were on high alert after a cam...,entailment,"Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy...","Your task is to tell, given a premise and a hy..."


In [145]:
print(test['Prompt for 10-shot'][2490])

Your task is to tell, given a premise and a hypothesis, if the hypothesis logically follows from the premise. I will give you the following examples to guide you:

- Premise: "On April 28 1945, at the Piazzale Loreto in Milan, Benito Mussolini and his mistress Clara Petacci were shot and hanged in a spectacle that was photographed repeatedly." hypothesis: "Mussolini's mistress was Clara Petacci." answer: "entailment"
- Premise: "The settlement must be approved by Citigroup's board of directors and the board of Regents of the University of California, the lead plaintiff for investors in the case." hypothesis: "The University of California's board of Regents still needs to ratify the settlement - as does Citigroup's board of directors." answer: "entailment"
- Premise: "White supported Gough Whitlam's Labor government of 1972 to 1975, and after Whitlam was ousted in the 1975 constitutional crisis, became particularly anti-royalist." hypothesis: "White assisted the Labor government from 19

In [146]:
test.to_csv("Prompted_Datasets/Entailment.csv")

# Dataset 9

This dataset encodes a task that is similar to the one encoded by dataset 8. Here, the software receives two sentences. Given the content of the first sentence, the software has to tell id the second sentence is true or not.

In [149]:
data = pd.read_excel("ChatGPT.xlsx", sheet_name=9)
data.head()

,ID,index,sentence1,sentence2,label,split,prompt,chatgpt_raw,chatgpt_final
0,1,0,I stuck a pin through a carrot. When I pulled ...,The carrot had a hole.,1,train,"Having the sentence ""I stuck a pin through a c...",1,1
1,2,1,John couldn't see the stage with Billy in fron...,John is so short.,1,train,"Having the sentence ""John couldn't see the sta...",1,1
2,3,2,The police arrested all of the gang members. T...,The police were trying to stop the drug trade ...,1,train,"Having the sentence ""The police arrested all o...",1,1
3,4,3,Steve follows Fred's example in everything. He...,Steve influences him hugely.,0,train,"Having the sentence ""Steve follows Fred's exam...",1,1
4,5,4,"When Tatyana reached the cabin, her mother was...","mother was careful not to disturb her, undress...",0,train,"Having the sentence ""When Tatyana reached the ...",0,0


In [150]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             852 non-null    int64 
 1   index          852 non-null    int64 
 2   sentence1      852 non-null    object
 3   sentence2      852 non-null    object
 4   label          852 non-null    object
 5   split          852 non-null    object
 6   prompt         852 non-null    object
 7   chatgpt_raw    852 non-null    object
 8   chatgpt_final  852 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 60.0+ KB


In [151]:
# Filtering out data with no label or not in training set
filtered = data[pd.notna(data['label']) & (data['split'] == 'train')][['sentence1','sentence2','label']]

# Sampling training examples (20 examples, 10 per class)
training_examples = stratified_sample_df(filtered, 'label', 10)
training_examples

,sentence1,sentence2,label
368,"Mary tucked her daughter Anne into bed, so tha...","Mary tucked her daughter Anne into bed, so tha...",0
272,Joe paid the detective after he delivered the ...,Joe delivered the final report on the case.,0
411,"I sallied out for a bit of food, more to pass ...","I sallied out for a bit of food, more to pass ...",0
15,I poured water from the bottle into the cup un...,The cup was empty.,0
221,Thomson visited Cooper's grave in 1765. At tha...,Thomson had been dead for five years.,0
252,The large ball crashed right through the table...,The table was made of steel.,0
517,"Beth didn't get angry with Sally, who had cut ...",Beth stopped and apologized.,0
343,Joe has sold his house and bought a new one a ...,He will be moving out of The new house on Thur...,0
459,The city councilmen refused the demonstrators ...,The demonstrators feared violence.,0
611,"There is a pillar between me and the stage, an...",I can't see around the pillar.,0


In [152]:
# Test set = dev split, as done by author et al. paper
test = data[pd.notna(data['label']) & (data['split'] == 'dev')][['sentence1','sentence2','label']]
test

,sentence1,sentence2,label
635,The drain is clogged with hair. It has to be c...,The hair has to be cleaned.,0
636,Jane knocked on Susan's door but she did not a...,Susan did not answer.,1
637,"Beth didn't get angry with Sally, who had cut ...",Sally stopped and counted to ten.,0
638,No one joins Facebook to be sad and lonely. Bu...,That's exactly how Facebook makes us feel.,1
639,The man couldn't lift his son because he was s...,The son was so heavy.,1
...,...,...,...
701,I poured water from the bottle into the cup un...,The bottle was empty.,1
702,Alice looked for her friend Jade in the crowd....,"Since Alice always wears a red turban, Alice s...",0
703,"The dog chased the cat, which ran up a tree. I...",The cat waited at the top.,1
704,"It was a summer afternoon, and the dog was sit...",The spot under the tree was hot.,0


In [154]:
def format_training_example(sentence1, sentence2, answer):
    return f"- First sentence: \"{sentence1}\" second sentence: \"{sentence2}\" answer: \"{answer}\""

def make_prompt(training_examples, sentence1, sentence2):
    return f"Your task is to tell, given the first sentence, if the second sentence is true or false. I will give you the following examples to guide you:\n\n{training_examples}\n\n\
Give your answer for the following sentences in just a python list without elaboration. Only one answer is admissible. Answer with 1 if the second sentence is true with respect to the first one and 0 otherwise.\
\nFirst sentence: {sentence1} second sentence: {sentence2}"

for k in range(1, 11):
    few_shot_examples = stratified_sample_df(training_examples, 'label', k).apply(lambda x: format_training_example(x['sentence1'], x['sentence2'], x['label']), axis = 1)
    examples_string = "\n".join(list(few_shot_examples))
    column_name = 'Prompt for ' + str(k) + '-shot'
    test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['sentence1'], x['sentence2']), axis = 1)

test.head()

,sentence1,sentence2,label,Prompt for 1-shot,Prompt for 2-shot,Prompt for 3-shot,Prompt for 4-shot,Prompt for 5-shot,Prompt for 6-shot,Prompt for 7-shot,Prompt for 8-shot,Prompt for 9-shot,Prompt for 10-shot
635,The drain is clogged with hair. It has to be c...,The hair has to be cleaned.,0,"Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence..."
636,Jane knocked on Susan's door but she did not a...,Susan did not answer.,1,"Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence..."
637,"Beth didn't get angry with Sally, who had cut ...",Sally stopped and counted to ten.,0,"Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence..."
638,No one joins Facebook to be sad and lonely. Bu...,That's exactly how Facebook makes us feel.,1,"Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence..."
639,The man couldn't lift his son because he was s...,The son was so heavy.,1,"Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence...","Your task is to tell, given the first sentence..."


In [156]:
print(test['Prompt for 10-shot'][635])

Your task is to tell, given the first sentence, if the second sentence is true or false. I will give you the following examples to guide you:

- First sentence: "The city councilmen refused the demonstrators a permit because they feared violence." second sentence: "The demonstrators feared violence." answer: "0"
- First sentence: "Joe paid the detective after he delivered the final report on the case." second sentence: "Joe delivered the final report on the case." answer: "0"
- First sentence: "The large ball crashed right through the table because it was made of steel." second sentence: "The table was made of steel." answer: "0"
- First sentence: "Mary tucked her daughter Anne into bed, so that she could sleep." second sentence: "Mary tucked her daughter Anne into bed, so that Mary could sleep." answer: "0"
- First sentence: "Joe has sold his house and bought a new one a few miles away. He will be moving out of it on Thursday." second sentence: "He will be moving out of The new house 

In [158]:
test.to_csv("Prompted_Datasets/WNLI.csv")

# Dataset 10

This is a dataset about question answering. The software receives a context and a question about the context. The software needs to answer the question with the context provided in input. If the context does not allow to answer the question, then the software should not answer.

In [160]:
data = pd.read_excel("ChatGPT.xlsx", sheet_name=10)
data.head()

,id,prompt,text,annotation,taken,chatgpt_raw,chatgpt_final
0,1,The context is'The Normans (Norman: Nourmands;...,The Normans (Norman: Nourmands; French: Norman...,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],...",1,"{ ""text"": [""Rollo""], ""answer_start"": [108] }","{ ""text"": [""Rollo""], ""answer_start"": [108] }"
1,2,The context is'The Normans (Norman: Nourmands;...,The Normans (Norman: Nourmands; French: Norman...,"{'text': [], 'answer_start': []}",2,"{ ""text"": [""Normandy""], ""answer_start"": [53] }","{ ""text"": [""Normandy""], ""answer_start"": [53] }"
2,3,The context is'The Norman dynasty had a major ...,"The Norman dynasty had a major political, cult...","{'text': ['William the Conqueror', 'William th...",3,"{ ""text"": [""William the Conqueror""], ""answer_s...","{ ""text"": [""William the Conqueror""], ""answer_s..."
3,4,The context is'The Norman dynasty had a major ...,"The Norman dynasty had a major political, cult...","{'text': [], 'answer_start': []}",4,"{ ""text"": [""political, cultural and military i...","{ ""text"": [""political, cultural and military i..."
4,5,The context is'The Norman dynasty had a major ...,"The Norman dynasty had a major political, cult...","{'text': [], 'answer_start': []}",5,"{ ""text"": [""Duke of Normandy""], ""answer_start""...","{ ""text"": [""Duke of Normandy""], ""answer_start""..."


In [161]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1000 non-null   int64 
 1   prompt         1000 non-null   object
 2   text           1000 non-null   object
 3   annotation     1000 non-null   object
 4   taken          1000 non-null   int64 
 5   chatgpt_raw    999 non-null    object
 6   chatgpt_final  1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [177]:
# Let's make a new column which tells if the question is answerable given the context, or not.
# This column will help us to sample the few shot examples to binclude in the prompts.

def is_answerable(x):
    return not eval(x['annotation'])['text'] == []

data['is_answerable'] = data.apply(is_answerable, axis = 1)

In [222]:
# Since blablabla et al. did not include the question as a column, we extract it from the prompt...

def extract_question(x):
    question_span = re.search(r"Answer to the question.*\'.*\'", x).span()
    question_block = x[question_span[0]:question_span[1]]
    question_span = re.search(r"\'.*\'", question_block).span()
    return question_block[question_span[0]+1:question_span[1]-1]

data['question'] = data['prompt'].apply(extract_question)

In [284]:
# To format the few shot examples, we extract the first annotation from all the possible annotations
def get_first_annotation(x):
    if x["is_answerable"]:
        return eval(x['annotation'])['text'][0]
    else:
        return "[]"

data['annotation_1'] = data.apply(get_first_annotation, axis = 1)

In [286]:
data

,id,prompt,text,annotation,taken,chatgpt_raw,chatgpt_final,is_answerable,question,annotation_1
0,1,The context is'The Normans (Norman: Nourmands;...,The Normans (Norman: Nourmands; French: Norman...,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],...",1,"{ ""text"": [""Rollo""], ""answer_start"": [108] }","{ ""text"": [""Rollo""], ""answer_start"": [108] }",True,Who was the Norse leader?,Rollo
1,2,The context is'The Normans (Norman: Nourmands;...,The Normans (Norman: Nourmands; French: Norman...,"{'text': [], 'answer_start': []}",2,"{ ""text"": [""Normandy""], ""answer_start"": [53] }","{ ""text"": [""Normandy""], ""answer_start"": [53] }",False,What is France a region of?,[]
2,3,The context is'The Norman dynasty had a major ...,"The Norman dynasty had a major political, cult...","{'text': ['William the Conqueror', 'William th...",3,"{ ""text"": [""William the Conqueror""], ""answer_s...","{ ""text"": [""William the Conqueror""], ""answer_s...",True,Who was the duke in the battle of Hastings?,William the Conqueror
3,4,The context is'The Norman dynasty had a major ...,"The Norman dynasty had a major political, cult...","{'text': [], 'answer_start': []}",4,"{ ""text"": [""political, cultural and military i...","{ ""text"": [""political, cultural and military i...",False,What type of major impact did the Norman dynas...,[]
4,5,The context is'The Norman dynasty had a major ...,"The Norman dynasty had a major political, cult...","{'text': [], 'answer_start': []}",5,"{ ""text"": [""Duke of Normandy""], ""answer_start""...","{ ""text"": [""Duke of Normandy""], ""answer_start""...",False,Who ruled the country of Normandy?,[]
...,...,...,...,...,...,...,...,...,...,...
995,996,The context is'Since all modern ctenophores ex...,Since all modern ctenophores except the beroid...,"{'text': ['Richard Harbison', 'Richard Harbiso...",996,"{ ""text"": [""Richard Harbison""], ""answer_start""...","{ ""text"": [""Richard Harbison""], ""answer_start""...",True,Who did a morphologically analysis in 1985 tha...,Richard Harbison
996,997,"The context is'Fresno (/ˈfrɛznoʊ/ FREZ-noh), t...","Fresno (/ˈfrɛznoʊ/ FREZ-noh), the county seat ...","{'text': ['Fresno', 'Fresno', 'Fresno'], 'answ...",997,"{ ""text"": [""Fresno""], ""answer_start"": [0] }","{ ""text"": [""Fresno""], ""answer_start"": [0] }",True,Which city is the fifth-largest city in Califo...,Fresno
997,998,"The context is'Fresno (/ˈfrɛznoʊ/ FREZ-noh), t...","Fresno (/ˈfrɛznoʊ/ FREZ-noh), the county seat ...","{'text': ['ash leaf', 'ash leaf', 'an ash leaf...",998,"{ ""text"": [""ash leaf""], ""answer_start"": [148] }","{ ""text"": [""ash leaf""], ""answer_start"": [148] }",True,What is featured on the city of Fresno's city ...,ash leaf
998,999,"The context is'In 1872, the Central Pacific Ra...","In 1872, the Central Pacific Railroad establis...","{'text': ['1885', '1885', '1885'], 'answer_sta...",999,"{ ""text"": [ ""1885"" ], ""answer_start"": [ 97 ] }","{ ""text"": [ ""1885"" ], ""answer_start"": [ 97 ] }",True,In what year did Fresno become an incorporated...,1885


In [287]:
# Filtering out irrelevant columns
filtered = data[['text','question','annotation','is_answerable','annotation_1']]

# Sampling training examples (20 examples, 10 per class). Here, the class is whether the question is answerable or not
training_examples = stratified_sample_df(filtered, 'is_answerable', 10)
training_examples

,text,question,annotation,is_answerable,annotation_1
408,"Prior to European settlement, the area now con...",Where was the first settlement in Canberra?,"{'text': [], 'answer_start': []}",False,[]
535,The first Huguenots to leave France sought fre...,How many were in the first wave of Huguenot's ...,"{'text': [], 'answer_start': []}",False,[]
296,Southern California is home to Los Angeles Int...,What is the busiest airport by passenger volum...,"{'text': [], 'answer_start': []}",False,[]
15,One of the first Norman mercenaries to serve a...,Who was the first Byzantine mercenary to serve...,"{'text': [], 'answer_start': []}",False,[]
471,Huguenot numbers peaked near an estimated two ...,How many French Catholics were there in 1598?,"{'text': [], 'answer_start': []}",False,[]
457,In 1854 at Ballarat there was an armed rebelli...,What did colonial authorities reduce because o...,"{'text': [], 'answer_start': []}",False,[]
395,"In November 2006, the Victorian Legislative Co...",In what month does the Victorian Parliament re...,"{'text': [], 'answer_start': []}",False,[]
202,Earlier papers studying problems solvable by T...,Who wrote a paper on real time computations in...,"{'text': [], 'answer_start': []}",False,[]
9,"Before Rollo's arrival, its populations did no...",What Viking groups were conquered by Rollo?,"{'text': [], 'answer_start': []}",False,[]
347,"On 18 November 2015, Sky announced Sky Q, a ra...",What are the Sky Q mini set top boxes never ab...,"{'text': [], 'answer_start': []}",False,[]


In [292]:
# Creating the test by excluding the examples taken for the training
test = filtered.iloc[filtered.index.difference(training_examples.index)]
test.head()

,text,question,annotation,is_answerable,annotation_1
0,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],...",True,Rollo
1,The Normans (Norman: Nourmands; French: Norman...,What is France a region of?,"{'text': [], 'answer_start': []}",False,[]
2,"The Norman dynasty had a major political, cult...",Who was the duke in the battle of Hastings?,"{'text': ['William the Conqueror', 'William th...",True,William the Conqueror
3,"The Norman dynasty had a major political, cult...",What type of major impact did the Norman dynas...,"{'text': [], 'answer_start': []}",False,[]
4,"The Norman dynasty had a major political, cult...",Who ruled the country of Normandy?,"{'text': [], 'answer_start': []}",False,[]


In [296]:
def format_training_example(context, question, answer):
    return f"- Context: \"{context}\" Question: \"{question}\" Answer: \"{answer}\""

def make_prompt(training_examples, context, question):
    return f"Your task is to asnwer a question about a given context paragraph, if the context paragraph provides enough information to answer the question. I will give you the following examples to guide you:\n\n{training_examples}\n\n" \
    f"Given the following context paragraph:\n\"{context}\"\n\nAnswer the following question with the smallest text span from the previous text, without any elaboration from you. Report the answer as a signle python string." \
    f"If the previous paragraph does not provide enough information to answer the following question, answer with an empty python list.\n\"{question}\""

for k in range(1, 11):
    few_shot_examples = stratified_sample_df(training_examples, 'is_answerable', k).apply(lambda x: format_training_example(x['text'], x['question'], x['annotation_1']), axis = 1)
    examples_string = "\n".join(list(few_shot_examples))
    column_name = 'Prompt for ' + str(k) + '-shot'
    test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['text'], x['question']), axis = 1)

test.head()

C:\Users\sap\AppData\Local\Temp\ipykernel_32532\2333557848.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['text'], x['question']), axis = 1)


,text,question,annotation,is_answerable,annotation_1,Prompt for 1-shot,Prompt for 2-shot,Prompt for 3-shot,Prompt for 4-shot,Prompt for 5-shot,Prompt for 6-shot,Prompt for 7-shot,Prompt for 8-shot,Prompt for 9-shot,Prompt for 10-shot
0,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],...",True,Rollo,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...
1,The Normans (Norman: Nourmands; French: Norman...,What is France a region of?,"{'text': [], 'answer_start': []}",False,[],Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...
2,"The Norman dynasty had a major political, cult...",Who was the duke in the battle of Hastings?,"{'text': ['William the Conqueror', 'William th...",True,William the Conqueror,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...
3,"The Norman dynasty had a major political, cult...",What type of major impact did the Norman dynas...,"{'text': [], 'answer_start': []}",False,[],Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...
4,"The Norman dynasty had a major political, cult...",Who ruled the country of Normandy?,"{'text': [], 'answer_start': []}",False,[],Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...,Your task is to asnwer a question about a give...


In [301]:
print(test['Prompt for 10-shot'][3])

Your task is to asnwer a question about a given context paragraph, if the context paragraph provides enough information to answer the question. I will give you the following examples to guide you:

- Context: "Before Rollo's arrival, its populations did not differ from Picardy or the Île-de-France, which were considered "Frankish". Earlier Viking settlers had begun arriving in the 880s, but were divided between colonies in the east (Roumois and Pays de Caux) around the low Seine valley and in the west in the Cotentin Peninsula, and were separated by traditional pagii, where the population remained about the same with almost no foreign settlers. Rollo's contingents who raided and ultimately settled Normandy and parts of the Atlantic coast included Danes, Norwegians, Norse–Gaels, Orkney Vikings, possibly Swedes, and Anglo-Danes from the English Danelaw under Norse control." Question: "What Viking groups were conquered by Rollo?" Answer: "[]"
- Context: "The first Huguenots to leave Franc

In [302]:
test.to_csv("Prompted_Datasets/SQuAD.csv")

# Dataset 11

This is a dataset about mathematical question answering. The software receives a math question and has to answer correctly.

In [306]:
data = pd.read_excel("ChatGPT.xlsx", sheet_name=11)
data.head()

,id,prompt,text,expected_res,chatgpt_raw,chatgpt_final
0,0,Write the reasoning and highlight the answer a...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18,"Reasoning: First, we need to find out how many...",18.0
1,1,Write the reasoning and highlight the answer a...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,3,Reasoning: The question tells us that a robe t...,3.0
2,2,Write the reasoning and highlight the answer a...,The cost of the house and repairs came out to ...,70000,"Reasoning: Josh buys a house for $80,000 and t...",10000.0
3,3,Write the reasoning and highlight the answer a...,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,540,Reasoning: James runs 3 sprints 3 times a week...,540.0
4,4,Write the reasoning and highlight the answer a...,"If each chicken eats 3 cups of feed per day, t...",20,Reasoning: Each chicken needs 3 cups of mixed ...,20.0


In [315]:
# Extracting the question from the prompt

def extract_question(x):
    start = x.find("The question is: ") + len("The question is: ")
    return x[start:]

data['question'] = data['prompt'].apply(extract_question)

In [316]:
data

,id,prompt,text,expected_res,chatgpt_raw,chatgpt_final,question
0,0,Write the reasoning and highlight the answer a...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18,"Reasoning: First, we need to find out how many...",18.00,Janet’s ducks lay 16 eggs per day. She eats th...
1,1,Write the reasoning and highlight the answer a...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,3,Reasoning: The question tells us that a robe t...,3.00,A robe takes 2 bolts of blue fiber and half th...
2,2,Write the reasoning and highlight the answer a...,The cost of the house and repairs came out to ...,70000,"Reasoning: Josh buys a house for $80,000 and t...",10000.00,Josh decides to try flipping a house. He buys...
3,3,Write the reasoning and highlight the answer a...,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,540,Reasoning: James runs 3 sprints 3 times a week...,540.00,James decides to run 3 sprints 3 times a week....
4,4,Write the reasoning and highlight the answer a...,"If each chicken eats 3 cups of feed per day, t...",20,Reasoning: Each chicken needs 3 cups of mixed ...,20.00,"Every day, Wendi feeds each of her chickens th..."
...,...,...,...,...,...,...,...
995,995,Write the reasoning and highlight the answer a...,Adam's father gave him $6 * 2 = $<<6*2=12>>12....,12,Adam's father gave him $6*2 = $12 for the trou...,12.00,Adam bought himself some new trousers for $30....
996,996,Write the reasoning and highlight the answer a...,"If the combined age of the children is 20, the...",3,"To find the age of Wilfred, we can use the inf...",0.00,"Four children are playing together—Akbar, Ales..."
997,997,Write the reasoning and highlight the answer a...,Walter collects $500 + $1500 + $200 = $<<500+1...,6600,To find out how much Walter's lawyer is going ...,6600.00,Walter is collecting money for charity. First ...
998,998,Write the reasoning and highlight the answer a...,If each of the 100 guests brought a plate of 1...,50,To find out how many people were in the church...,50.00,Dylan attended a wedding where there were 100 ...


In [327]:
# Filtering out irrelevant columns
filtered = data[['text','question','expected_res']]

# Sampling training examples (10 random training examples for the few shot)
training_examples = filtered.sample(10, random_state=42)
training_examples

,text,question,expected_res
521,On Tuesday he completes 3 * 12 = <<3*12=36>>36...,Chris is way behind on his math homework. He h...,39
737,The giant cubes used up 12 ounces of water bec...,Peter wants to make different sized ice cubes ...,2
740,The number of students that like to play music...,A class of 50 students has various hobbies. 10...,25
660,Let g be the number of geese and let s be the ...,Lee rears only sheep and geese on his farm. I...,15
411,The second package was worth 400*2=$<<400*2=80...,Tim decides to light off some fireworks for th...,1110
678,Karen earns $10 x 2 = $<<10*2=20>>20 per hour....,Lorie earns $10 per hour. Karen earns twice wh...,120
626,"With a speed of 20 kb per second, there are 20...",Ashley has an internet connection speed of 20k...,72
513,Julie spend $500 * 0.2 = $<<500*0.2=100>>100 f...,Julie had $500. She spent 20% of it on clothes...,300
859,The total amount of money that Rafaela receive...,"While working at the restaurant, each of the f...",1520
136,Scottish Unicorns:27(1/3)=9\nFemale:9(2/3)=6 u...,There are 27 unicorns left in the world. One ...,6


In [328]:
# Creating the test by excluding the examples taken for the training
test = filtered.iloc[filtered.index.difference(training_examples.index)]
test.head()

,text,question,expected_res
0,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,Janet’s ducks lay 16 eggs per day. She eats th...,18
1,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,A robe takes 2 bolts of blue fiber and half th...,3
2,The cost of the house and repairs came out to ...,Josh decides to try flipping a house. He buys...,70000
3,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,James decides to run 3 sprints 3 times a week....,540
4,"If each chicken eats 3 cups of feed per day, t...","Every day, Wendi feeds each of her chickens th...",20


In [339]:
def format_training_example(problem, solution_process, result):
    return f"- Problem: \"{problem}\" Solution process: \"{solution_process}\" The solution is {result}"

def make_prompt(training_examples, question):
    return f"Your task is to solve a mathematical problem. I will give you the following examples to guide you:\n\n{training_examples}\n\n" \
        f"Solve the following problem. Please, report your calculations. At the end of your calculations, provide a final sentence of the form \"The solution to this problem is X\", " \
        f"where X is the correct number. In the final sentence, provide only the correct number, without any other explanation.\n\"{question}\""

for k in range(1, 11):
    few_shot_examples = training_examples.sample(k, random_state=42).apply(lambda x: format_training_example(x['question'], x['text'], x["expected_res"]), axis = 1)
    examples_string = "\n".join(list(few_shot_examples))
    column_name = 'Prompt for ' + str(k) + '-shot'
    test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['question']), axis = 1)

test.head()

C:\Users\sap\AppData\Local\Temp\ipykernel_32532\3874514158.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['question']), axis = 1)


,text,question,expected_res,Prompt for 1-shot,Prompt for 2-shot,Prompt for 3-shot,Prompt for 4-shot,Prompt for 5-shot,Prompt for 6-shot,Prompt for 7-shot,Prompt for 8-shot,Prompt for 9-shot,Prompt for 10-shot
0,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,Janet’s ducks lay 16 eggs per day. She eats th...,18,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...
1,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,A robe takes 2 bolts of blue fiber and half th...,3,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...
2,The cost of the house and repairs came out to ...,Josh decides to try flipping a house. He buys...,70000,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...
3,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,James decides to run 3 sprints 3 times a week....,540,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...
4,"If each chicken eats 3 cups of feed per day, t...","Every day, Wendi feeds each of her chickens th...",20,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...,Your task is to solve a mathematical problem. ...


In [340]:
print(test["Prompt for 10-shot"][0])

Your task is to solve a mathematical problem. I will give you the following examples to guide you:

- Problem: "While working at the restaurant, each of the forty customers who came into the restaurant gave Rafaela a $20 tip. Julieta received 10% less money in tips than Rafaela. How much money did Julieta and Rafaela receive as tips altogether?" Solution process: "The total amount of money that Rafaela received as tips from the forty customers is 40*$20 = $<<40*20=800>>800
Julieta got 10% less money in tips than Rafaela, a total of 10/100*$800 = $<<10/100*800=80>>80
The total amount of money that Julieta received as tips is $800-$80 = $<<800-80=720>>720
Altogether, Julieta, and Rafaela received $720+$800 = $<<720+800=1520>>1520
#### 1520" The solution is 1520
- Problem: "Peter wants to make different sized ice cubes with 32 ounces of water. He can make giant cubes that use 4 ounces per cube, medium cubes that use 2 ounces, and small cubes that use 1/2 an ounce. If he makes 3 giant cube

In [341]:
test.to_csv("Prompted_Datasets/MathQA.csv")

# Dataset 13

This dataset pertains multiclass, multilabel emotion classification.

In [349]:
data = pd.read_excel("ChatGPT.xlsx", sheet_name=13)
data.head()

,id,prompt,text,annotation,taken,chatgpt_final,chatgpt_raw,comment
0,1,"From the given list of all emotions, choose th...",NaN,sadness,1,"[""sadness""]","[""sadness""]",NaN
1,2,"From the given list of all emotions, choose th...",NaN,admiration,1,"[""confusion""]","[""confusion""]",NaN
2,3,"From the given list of all emotions, choose th...",NaN,excitement,1,"[""neutral""]","[""neutral""]",NaN
3,4,"From the given list of all emotions, choose th...",NaN,gratitude,1,"[""gratitude""]","[""gratitude""]",NaN
4,5,"From the given list of all emotions, choose th...",NaN,neutral,1,"[""neutral""]","[""neutral""]",NaN


In [458]:
# Extracting the sentence from the prompt

def extract_question(x):
    start = x.find("Input text: ") + len("Input text: ")
    return x[start:]

data['sentence'] = data['prompt'].apply(extract_question)

In [459]:
data

,id,prompt,text,annotation,taken,chatgpt_final,chatgpt_raw,comment,question,sentence
0,1,"From the given list of all emotions, choose th...",NaN,sadness,1,"[""sadness""]","[""sadness""]",NaN,I’m really sorry about your situation :( Altho...,I’m really sorry about your situation :( Altho...
1,2,"From the given list of all emotions, choose th...",NaN,admiration,1,"[""confusion""]","[""confusion""]",NaN,It's wonderful because it's awful. At not with.,It's wonderful because it's awful. At not with.
2,3,"From the given list of all emotions, choose th...",NaN,excitement,1,"[""neutral""]","[""neutral""]",NaN,"Kings fan here, good luck to you guys! Will be...","Kings fan here, good luck to you guys! Will be..."
3,4,"From the given list of all emotions, choose th...",NaN,gratitude,1,"[""gratitude""]","[""gratitude""]",NaN,"I didn't know that, thank you for teaching me ...","I didn't know that, thank you for teaching me ..."
4,5,"From the given list of all emotions, choose th...",NaN,neutral,1,"[""neutral""]","[""neutral""]",NaN,They got bored from haunting earth for thousan...,They got bored from haunting earth for thousan...
...,...,...,...,...,...,...,...,...,...,...
995,996,"From the given list of all emotions, choose th...",NaN,disapproval,1,"[""disapproval""]","[""disapproval""]",NaN,"says you, too bad you're totally wrong.","says you, too bad you're totally wrong."
996,997,"From the given list of all emotions, choose th...",NaN,gratitude,1,"[""approval""]","[""approval""]",NaN,Thanks for giving advice to the people who nee...,Thanks for giving advice to the people who nee...
997,998,"From the given list of all emotions, choose th...",NaN,neutral,1,"[""disapproval""]","[""disapproval""]",NaN,Where did I state I wanted to keep Syria going...,Where did I state I wanted to keep Syria going...
998,999,"From the given list of all emotions, choose th...",NaN,admiration,1,"[""approval""]","[""approval""]",NaN,Good kid.,Good kid.


In [473]:
# In author et al., they also tell ChatGPT how many words to use for the annotation. It would be good to have ChatGPT coiming up with all the most pertinent tags on its own.
print(data['prompt'][0]) # This asks for "exactly 1 selected most matching emotion"
print(data['prompt'][13]) # This asks for "exactly 2 selected most matching emotions"

From the given list of all emotions, choose the ones that the input text arouses in most people reading it. Write your answer in the form of a Python list containing exactly 1 selected most matching emotion. List of all emotions: admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, excitement, fear, gratitude, grief, joy, love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, neutral. Input text: I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!
From the given list of all emotions, choose the ones that the input text arouses in most people reading it. Write your answer in the form of a Python list containing exactly 2 selected most matching emotions. List of all emotions: admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, exciteme

In [460]:
# Not enough samples with more than 1 annotation. Therefore, to select the few shot examples, we will sample from rows with only one emotion annotation.
data[data['annotation'].apply(lambda x: len(x.split(','))) > 1][['annotation', 'id']].groupby('annotation').count().sort_values('id', ascending = False)

,id
annotation,
"realization,neutral",6
"anger,annoyance",6
"curiosity,neutral",5
"approval,neutral",4
"annoyance,neutral",3
...,...
"annoyance,disapproval,disgust",1
"annoyance,disappointment",1
"annoyance,curiosity",1


In [461]:
# Emotions such as grief and pride appear with only one example. Therefore, the only few-shot example we can prompt here is 1 example per class.
data[data['annotation'].apply(lambda x: len(x.split(','))) == 1][['annotation','id']].groupby('annotation').count().sort_values('id', ascending = False)

,id
annotation,
neutral,301
gratitude,62
admiration,55
disapproval,42
annoyance,35
amusement,32
anger,31
approval,30
curiosity,26


In [462]:
def unique_label_stratified_sample_df(df, col, n_samples, random_state = 42):
    df_f = df[df[col].apply(lambda x: len(x.split(','))) == 1]
    n = min(n_samples, df_f[col].value_counts().min())
    print(n)
    df_ = df_f.groupby(col).apply(lambda x: x.sample(n, random_state = random_state))
    df_.index = df_.index.droplevel(0)
    return df_

In [465]:
# Filtering out irrelevant columns
filtered = data[['sentence','annotation']]

# As said before, we sample 1 example per class, giving us 28 examples in total.
training_examples = unique_label_stratified_sample_df(filtered, 'annotation', 1)
training_examples

1


,sentence,annotation
627,But they are spectacular.,admiration
965,"oh.. actual physical mail, hehehe I hear ya",amusement
855,"""Moose killed my parents.""",anger
800,they already ruined their own lives by having ...,annoyance
958,This is too accurate 😂,approval
41,please dont bring a woman in as a sex toy for ...,caring
85,I'm not sure I have heard of this. Really inte...,confusion
466,"Has he done this with other world leaders, too...",curiosity
543,Now we need [NAME] to reunite with [NAME],desire
638,Idiots are downvoting your correct comment.,disappointment


In [466]:
# Creating the test by excluding the examples taken for the training
test = filtered.iloc[filtered.index.difference(training_examples.index)]
test.head()

,sentence,annotation
0,I’m really sorry about your situation :( Altho...,sadness
1,It's wonderful because it's awful. At not with.,admiration
2,"Kings fan here, good luck to you guys! Will be...",excitement
3,"I didn't know that, thank you for teaching me ...",gratitude
4,They got bored from haunting earth for thousan...,neutral


In [467]:
def format_training_example(sentence, annotation):
    return f"- Sentence: \"{sentence}\" Annotation: \"{annotation}\""

def make_prompt(training_examples, text):
    return f"Your task is to classify an input text according to the emotions it contains. I will provide you with a list of emotions from which you can choose the words which best describe the text." \
        f"The list of emotions is the following: [admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, excitement, fear, gratitude, grief, " \
        f"joy, love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, neutral].\n\n" \
        f"I will give you the following examples to guide you:\n\n{training_examples}\n\n" \
        f"The text to classify is the following. Please, choose the most appropriate words from the list of emotions and answer with a python list, without any elaboration from you:\n{text}"

few_shot_examples = training_examples.apply(lambda x: format_training_example(x['sentence'], x['annotation']), axis = 1)
examples_string = "\n".join(list(few_shot_examples))
column_name = 'Prompt for 1-shot'
test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['sentence']), axis = 1)

test.head()

C:\Users\sap\AppData\Local\Temp\ipykernel_32532\1001005093.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['sentence']), axis = 1)


,sentence,annotation,Prompt for 1-shot
0,I’m really sorry about your situation :( Altho...,sadness,Your task is to classify an input text accordi...
1,It's wonderful because it's awful. At not with.,admiration,Your task is to classify an input text accordi...
2,"Kings fan here, good luck to you guys! Will be...",excitement,Your task is to classify an input text accordi...
3,"I didn't know that, thank you for teaching me ...",gratitude,Your task is to classify an input text accordi...
4,They got bored from haunting earth for thousan...,neutral,Your task is to classify an input text accordi...


In [468]:
print(test['Prompt for 1-shot'][0])

Your task is to classify an input text according to the emotions it contains. I will provide you with a list of emotions from which you can choose the words which best describe the text.The list of emotions is the following: [admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, excitement, fear, gratitude, grief, joy, love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, neutral].

I will give you the following examples to guide you:

- Sentence: "But they are spectacular." Annotation: "admiration"
- Sentence: "oh.. actual physical mail, hehehe I hear ya" Annotation: "amusement"
- Sentence: ""Moose killed my parents."" Annotation: "anger"
- Sentence: "they already ruined their own lives by having an affair. the other spouse knowing is just a consequence of their selfish actions." Annotation: "annoyance"
- Sentence: "This is too accurate 😂" Annotation: "approval"
- Sen

In [469]:
test.to_csv("Prompted_Datasets/GoEmo.csv")

# Dataset 13: new approach
With the previous approach, we cannot go beyond the 1-shot. To overcome the limitations we take another road: we get the original training set of the GoEmo Dataset and we extract enough examples from it. Then, we prompt the same 1000 examples selected by the authors of "Jack of all trades master of none".

In [107]:
# Getting the original datasets
!wget https://github.com/google-research/google-research/raw/master/goemotions/data/test.tsv
!wget https://github.com/google-research/google-research/raw/master/goemotions/data/train.tsv
!wget https://github.com/google-research/google-research/raw/master/goemotions/data/dev.tsv


--2024-03-25 12:24:50--  https://github.com/google-research/google-research/raw/master/goemotions/data/test.tsv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/test.tsv [following]
--2024-03-25 12:24:50--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 436706 (426K) [text/plain]
Saving to: 'test.tsv.1'

     0K .......... .......... .......... .......... .......... 11% 1.70M 0s
    50K .......... .......... .......... .......... .......... 23% 

In [108]:
# Getting the labels
!wget https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/emotions.txt

--2024-03-25 12:24:55--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/emotions.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248 [text/plain]
Saving to: 'emotions.txt.1'

     0K                                                       100% 9.27M=0s

2024-03-25 12:24:55 (9.27 MB/s) - 'emotions.txt.1' saved [248/248]



In [109]:
labels_df = pd.read_csv('emotions.txt', header=None)
labels = labels_df.values[:,0].tolist()

In [139]:
labels_df

,0
0,admiration
1,amusement
2,anger
3,annoyance
4,approval
5,caring
6,confusion
7,curiosity
8,desire
9,disappointment


In [110]:
train_df = pd.read_csv('train.tsv', sep='\t', header=None)

In [111]:
train_df.head()

,0,1,2
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj


In [112]:
del train_df[2]

In [113]:
train_df.head()

,0,1
0,My favourite food is anything I didn't have to...,27
1,"Now if he does off himself, everyone will thin...",27
2,WHY THE FUCK IS BAYLESS ISOING,2
3,To make her feel threatened,14
4,Dirty Southern Wankers,3


In [114]:
train_df.columns = ['sentence', 'num_labels']

In [115]:
# This function takes in input the numerical labels of the GoEmo dataset and returns a list of textual labels
def text_labels(labs):
    return ",".join(list(map(labels.__getitem__, map(int, labs.split(',')))))

In [116]:
train_df['labels'] = train_df['num_labels'].apply(text_labels)

In [117]:
train_df.head()

,sentence,num_labels,labels
0,My favourite food is anything I didn't have to...,27,neutral
1,"Now if he does off himself, everyone will thin...",27,neutral
2,WHY THE FUCK IS BAYLESS ISOING,2,anger
3,To make her feel threatened,14,fear
4,Dirty Southern Wankers,3,annoyance


In [118]:
# An example with multiple labels
train_df.iloc[[11900]]

,sentence,num_labels,labels
11900,LOL at the tldr. What a crazy story. I'm glad ...,"0,1,17","admiration,amusement,joy"


In [119]:
# Here we have enough data for the 10-shot
# We still note that the most common label is "neutral"
train_df[train_df['labels'].apply(lambda x: len(x.split(','))) == 1][['labels']].value_counts()

labels        
neutral           12823
admiration         2710
approval           1873
gratitude          1857
amusement          1652
annoyance          1451
love               1427
disapproval        1402
curiosity          1389
anger              1025
optimism            861
confusion           858
joy                 853
sadness             817
surprise            720
disappointment      709
caring              649
realization         586
excitement          510
disgust             498
fear                430
desire              389
remorse             353
embarrassment       203
relief               88
nervousness          85
pride                51
grief                39
Name: count, dtype: int64

In [120]:
def unique_label_stratified_sample_df(df, col, n_samples, random_state = 42):
    df_f = df[df[col].apply(lambda x: len(x.split(','))) == 1]
    n = min(n_samples, df_f[col].value_counts().min())
    print(n)
    df_ = df_f.groupby(col).apply(lambda x: x.sample(n, random_state = random_state))
    df_.index = df_.index.droplevel(0)
    return df_

In [121]:
# Filtering out irrelevant columns
filtered = train_df[['sentence','labels']]

# As said before, we sample 10 example per class, giving us 280 examples in total.
training_examples = unique_label_stratified_sample_df(filtered, 'labels', 10)
training_examples

10


,sentence,labels
42995,"Wow, amazing job, Mr. [NAME]! Another liberal ...",admiration
39527,"One good thing about vaping When it hits you, ...",admiration
7580,That is an awesome idea and execution. Also a ...,admiration
4625,Hockey players are more brave than the US Mari...,admiration
21640,What a good pupper!,admiration
...,...,...
3727,I can’t believe that’s real,surprise
3618,Wow random irrelevant internet person has the ...,surprise
9385,Wow City all of a sudden not so invincible any...,surprise
18122,really thought he'd hit a bird for the longest...,surprise


In [122]:
training_examples

,sentence,labels
42995,"Wow, amazing job, Mr. [NAME]! Another liberal ...",admiration
39527,"One good thing about vaping When it hits you, ...",admiration
7580,That is an awesome idea and execution. Also a ...,admiration
4625,Hockey players are more brave than the US Mari...,admiration
21640,What a good pupper!,admiration
...,...,...
3727,I can’t believe that’s real,surprise
3618,Wow random irrelevant internet person has the ...,surprise
9385,Wow City all of a sudden not so invincible any...,surprise
18122,really thought he'd hit a bird for the longest...,surprise


In [123]:
data = pd.read_excel("../../dataset/ChatGPT Evaluation v2.0.xlsx", sheet_name=13)
data.head()

,id,prompt,text,annotation,taken,chatgpt_final,chatgpt_raw,comment
0,1,"From the given list of all emotions, choose th...",NaN,sadness,1,"[""sadness""]","[""sadness""]",NaN
1,2,"From the given list of all emotions, choose th...",NaN,admiration,1,"[""confusion""]","[""confusion""]",NaN
2,3,"From the given list of all emotions, choose th...",NaN,excitement,1,"[""neutral""]","[""neutral""]",NaN
3,4,"From the given list of all emotions, choose th...",NaN,gratitude,1,"[""gratitude""]","[""gratitude""]",NaN
4,5,"From the given list of all emotions, choose th...",NaN,neutral,1,"[""neutral""]","[""neutral""]",NaN


In [124]:
# Extracting the sentence from the prompt

def extract_question(x):
    start = x.find("Input text: ") + len("Input text: ")
    return x[start:]

data['sentence'] = data['prompt'].apply(extract_question)

In [125]:
data.head()

,id,prompt,text,annotation,taken,chatgpt_final,chatgpt_raw,comment,sentence
0,1,"From the given list of all emotions, choose th...",NaN,sadness,1,"[""sadness""]","[""sadness""]",NaN,I’m really sorry about your situation :( Altho...
1,2,"From the given list of all emotions, choose th...",NaN,admiration,1,"[""confusion""]","[""confusion""]",NaN,It's wonderful because it's awful. At not with.
2,3,"From the given list of all emotions, choose th...",NaN,excitement,1,"[""neutral""]","[""neutral""]",NaN,"Kings fan here, good luck to you guys! Will be..."
3,4,"From the given list of all emotions, choose th...",NaN,gratitude,1,"[""gratitude""]","[""gratitude""]",NaN,"I didn't know that, thank you for teaching me ..."
4,5,"From the given list of all emotions, choose th...",NaN,neutral,1,"[""neutral""]","[""neutral""]",NaN,They got bored from haunting earth for thousan...


In [126]:
def format_training_example(sentence, annotation):
    return f"- Sentence: \"{sentence}\" Annotation: \"{annotation}\""

def make_prompt(training_examples, text):
    return f"Your task is to classify an input text according to the emotions it contains. I will provide you with a list of emotions from which you can choose the words which best describe the text." \
        f"The list of emotions is the following: [admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, excitement, fear, gratitude, grief, " \
        f"joy, love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, neutral].\n\n" \
        f"I will give you the following examples to guide you:\n\n{training_examples}\n\n" \
        f"The text to classify is the following. Please, choose the most appropriate words from the list of emotions and answer with a python list, without any elaboration from you:\n{text}"

In [127]:
def stratified_sample_df(df, col, n_samples, random_state = 42):
    n = min(n_samples, df[col].value_counts().min())
    df_ = df.groupby(col).apply(lambda x: x.sample(n, random_state = random_state))
    df_.index = df_.index.droplevel(0)
    return df_

In [128]:
format_training_example(training_examples['sentence'][42995], training_examples['labels'][42995])

'- Sentence: "Wow, amazing job, Mr. [NAME]! Another liberal absolutely DESTROYED with facts and logic! So eloquent and wise..." Annotation: "admiration"'

In [132]:
# Filtering out irrelevant columns
test = data[['sentence','annotation']]

In [133]:
for k in range(1, 11):
    few_shot_examples = stratified_sample_df(training_examples, 'labels', k).apply(lambda x: format_training_example(x['sentence'], x['labels']), axis = 1)
    examples_string = "\n".join(list(few_shot_examples))
    column_name = 'Prompt for ' + str(k) + '-shot'
    test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['sentence']), axis = 1)

test.head()


C:\Users\sap\AppData\Local\Temp\ipykernel_16552\4115397756.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['sentence']), axis = 1)
C:\Users\sap\AppData\Local\Temp\ipykernel_16552\4115397756.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[column_name] = test.apply(lambda x: make_prompt(examples_string, x['sentence']), axis = 1)
C:\Users\sap\AppData\Local\Temp\ipykernel_16552\4115397756.py:5: SettingWithCopyWarning: 
A value 

,sentence,annotation,Prompt for 1-shot,Prompt for 2-shot,Prompt for 3-shot,Prompt for 4-shot,Prompt for 5-shot,Prompt for 6-shot,Prompt for 7-shot,Prompt for 8-shot,Prompt for 9-shot,Prompt for 10-shot
0,I’m really sorry about your situation :( Altho...,sadness,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
1,It's wonderful because it's awful. At not with.,admiration,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
2,"Kings fan here, good luck to you guys! Will be...",excitement,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
3,"I didn't know that, thank you for teaching me ...",gratitude,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
4,They got bored from haunting earth for thousan...,neutral,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...


In [135]:
test.to_csv("13_GoEmo_New.csv")

In [136]:
test

,sentence,annotation,Prompt for 1-shot,Prompt for 2-shot,Prompt for 3-shot,Prompt for 4-shot,Prompt for 5-shot,Prompt for 6-shot,Prompt for 7-shot,Prompt for 8-shot,Prompt for 9-shot,Prompt for 10-shot
0,I’m really sorry about your situation :( Altho...,sadness,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
1,It's wonderful because it's awful. At not with.,admiration,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
2,"Kings fan here, good luck to you guys! Will be...",excitement,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
3,"I didn't know that, thank you for teaching me ...",gratitude,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
4,They got bored from haunting earth for thousan...,neutral,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
...,...,...,...,...,...,...,...,...,...,...,...,...
995,"says you, too bad you're totally wrong.",disapproval,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...
996,Thanks for giving advice to the people who nee...,gratitude,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is to classify an input text accordi...,Your task is t

In [138]:
print(test['Prompt for 1-shot'][0])

Your task is to classify an input text according to the emotions it contains. I will provide you with a list of emotions from which you can choose the words which best describe the text.The list of emotions is the following: [admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, excitement, fear, gratitude, grief, joy, love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, neutral].

I will give you the following examples to guide you:

- Sentence: "Fair enough." Annotation: "admiration"
- Sentence: "Again.. lolz." Annotation: "amusement"
- Sentence: ""If you don't support no go zones, you are racist!"" Annotation: "anger"
- Sentence: "A track would be much more suitable for this kind of recklessness! Keep this kind of dangerous driving off of public roads!" Annotation: "annoyance"
- Sentence: "Oh sweet! I'm a bit farther but go into the city all the time. I'm around 50